August: Calculating the V transport across the sections on the shelf break plane.

In [14]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline

import sys
sys.path.append('/ocean/imachuca/Canyons/analysis_mackenzie_canyon/notebooks/transport_fluxes/')
import planes_functions
sys.path.append('/ocean/imachuca/Canyons/analysis_mackenzie_canyon/notebooks/general_circulation/')
import general_functions

from salishsea_tools.nc_tools import scDataset

# $Q_j = v \times e1v \times e3v = v \times A_j$

In [15]:
import importlib
importlib.reload(planes_functions)

<module 'planes_functions' from '/ocean/imachuca/Canyons/analysis_mackenzie_canyon/notebooks/transport_fluxes/planes_functions.py'>

In [16]:
dirname = '/ocean/imachuca/Canyons/results_mackenzie/idealized/sbcana_forcings/agrif_wind_timestep/'
filename = '1_mesh_mask.nc'

gdepv, vmask, mbathy, e1v, e3v_0 = planes_functions.get_variables_V(dirname, filename)

In [18]:
ind_plane, ind_shelf, ind_bottom, ind_axis, ind_rimL, ind_rimR, ind_half, ind_shfL, ind_shfR,\
depth_shelf, depth_bottom, depth_half, area_j = planes_functions.get_indices_V(gdepv, vmask, mbathy, e1v, e3v_0)

ind_plane 178
ind_shelf 10
depth_shelf 91.875
ind_bottom 60
depth_bottom 534.672
ind_axis 110
ind_rimL 48
ind_rimR 172
ind_half 25
depth_half 223.125
ind_shfL 10
ind_shfR 210


In [19]:
def get_vars(dirname, filepattern, ind_plane, vmask):
    files = general_functions.get_files(dirname, filepattern, 'grid_V')
    x, y =  slice(1,-1,None), int(ind_plane)
    with scDataset(files) as ds:
        vomecrty0 = ds.variables['vomecrty'][:, :, y, x]
    vmask0 = vmask[:, y, :]
    vmask = np.tile(vmask0, (vomecrty0.shape[0],1, 1))  
    vomecrty = np.ma.array(vomecrty0, mask=1 - vmask)
    return vomecrty
# ------------------------------------------------------------------------------------------------

def calculate_flux_V(time_ind, velocity, mask, ind_plane, area):
    '''Calculates the flux through all cells along the shelf break
    cross-section for one instance in time.
    -----------------------------
    velocity (t, z, x)
    mask(z, y, x)
    area (z, x)
    Q (z, x)
    vel_masked (z, x)
    '''
    
    vel_plane = velocity[time_ind, :, :]
    vel_mask_plane = 1 - mask[:, ind_plane, :]
    vel_masked = np.ma.array(vel_plane, mask=vel_mask_plane)
    Q = vel_masked * area
    return Q, vel_masked

In [20]:
filepattern = "1_MCKNZ_1h_20170101_20170120_*"
vomecrty = get_vars(dirname, filepattern, ind_plane, vmask)

In [21]:
time_ind = 50
Q, vel_masked = calculate_flux_V(time_ind, vomecrty, vmask, ind_plane, area_j)